## Sales hedonics estimation

Paul Waddell, August 2015

In [1]:
%load_ext autoreload
%autoreload 2
if 'sim' not in globals():
    import os; os.chdir('..')
import models
import orca
import pandas as pd
pd.set_option('display.max_columns', 500)

### 1. Look at the Home Sales data

In [2]:
cl = orca.get_table('homesales').to_frame()
cl[1:5]

Filling column _node_id with value 91839.0 (4 values)
Filling column shape_area with value 604.350724409 (0 values)
Filling column zone_id with value 1178 (0 values)


,parcel_id,development_type_id,improvement_value,residential_units,residential_sqft,sqft_per_unit,non_residential_sqft,building_sqft,nonres_rent_per_sqft,res_price_per_sqft,stories,year_built,redfin_sale_price,redfin_sale_year,redfin_home_type,costar_property_type,costar_rent,building_type_id,price_per_sqft,node_id,zone_id,base_price_per_sqft
building_id,,,,,,,,,,,,,,,,,,,,,,
2,1623180,1,190994.72,1,1231,1231,0,1231,0,187.587598,1,1945,365000,2002,Single Family Residential,,,1,296.506905,157442,630,256.656665
3,954928,2,0.00,0,0,3042,0,0,0,0.000000,1,1962,1500000,2005,Multi-Family (2-4 Unit),,,3,493.096647,313439,183,375.000000
4,1192788,1,499852.00,1,2700,2700,0,2700,0,409.659023,1,1962,475000,1995,Single Family Residential,,,1,175.925926,164771,340,466.279147
5,1197767,1,53861.00,1,1200,1200,0,1200,0,436.304106,1,1967,380000,2000,Single Family Residential,,,1,316.666667,148653,296,309.871394


In [3]:
cl.describe()

,parcel_id,development_type_id,improvement_value,residential_units,residential_sqft,sqft_per_unit,non_residential_sqft,building_sqft,nonres_rent_per_sqft,res_price_per_sqft,stories,year_built,redfin_sale_price,redfin_sale_year,building_type_id,price_per_sqft,node_id,zone_id,base_price_per_sqft
count,1108514.000000,1108514.000000,1.108514e+06,1108514.000000,1108514.000000,1108514.000000,1108514.000000,1108514.000000,1108514.000000,1108514.000000,1108514.000000,1108514.000000,1.108514e+06,1108514.000000,1108514.000000,1108514.000000,1108514.000000,1108514.000000,1108514.000000
mean,1015268.710162,1.247653,2.934179e+05,1.513534,2586.475967,1797.644929,56.421703,2656.054463,0.163878,300.550859,1.220679,1968.361533,5.250805e+05,2002.854221,1.314444,314.660964,123542.438185,835.152881,282.383200
std,601958.232775,1.759316,3.406191e+06,5.596122,17723.199316,961.572536,2945.144870,18138.102387,1.523528,1393.318918,0.469785,25.705203,7.081612e+05,7.131412,1.341403,546.363958,87111.945754,399.161606,141.213638
min,114.000000,-1.000000,0.000000e+00,0.000000,0.000000,202.168317,0.000000,0.000000,0.000000,0.000000,1.000000,1805.000000,0.000000e+00,1962.000000,0.000000,0.000000,55.000000,1.000000,27.442371
25%,475342.250000,1.000000,1.339521e+05,1.000000,1280.000000,1200.000000,0.000000,1301.000000,0.000000,163.303174,1.000000,1953.000000,2.385000e+05,1998.000000,1.000000,149.450914,55727.250000,515.000000,200.171836
50%,1010044.000000,1.000000,2.118480e+05,1.000000,1763.000000,1594.000000,0.000000,1780.000000,0.000000,256.315409,1.000000,1970.000000,3.990000e+05,2004.000000,1.000000,243.111831,117932.000000,840.000000,262.500000
75%,1560265.750000,1.000000,3.233938e+05,1.000000,2471.000000,2164.000000,0.000000,2488.000000,0.000000,370.463909,1.000000,1989.000000,6.500000e+05,2009.000000,1.000000,381.591119,167305.000000,1185.000000,337.500000
max,2048547.000000,24.000000,3.355514e+09,1914.000000,8790000.000000,30000.000000,1479817.000000,8790000.000000,64.373133,1445456.524959,59.000000,2015.000000,1.800000e+08,2014.000000,16.000000,278582.488387,354681.000000,1454.000000,16895.694918


### 2. Set up the network vars

In [4]:
%%capture
orca.run(["neighborhood_vars"])

### 3. Estimate a sales hedonic using Redfin sales transactions

In [6]:
# The model expression is in rsh.yaml; price_per_sqft is the sale price per square 
# foot from the Redfin homesales. Price, sqft, and bedrooms are specific to the unit, 
# while all the other variables are aggregations at the node or zone level. Note that we 
# can't use bedrooms in the simulation stage because it's not in the unit data.

In [17]:
orca.run(["rsh_estimate"])

Running step 'rsh_estimate'
                              OLS Regression Results                              
Dep. Variable:     np.log(price_per_sqft)   R-squared:                       0.433
Model:                                OLS   Adj. R-squared:                  0.433
Method:                     Least Squares   F-statistic:                     5395.
Date:                    Sat, 05 Sep 2015   Prob (F-statistic):               0.00
Time:                            08:31:16   Log-Likelihood:                -29454.
No. Observations:                  106000   AIC:                         5.894e+04
Df Residuals:                      105984   BIC:                         5.909e+04
Df Model:                              15                                         
Covariance Type:                nonrobust                                         
                                   coef    std err          t      P>|t|      [95.0% Conf. Int.]
---------------------------------------------

In [ ]:
# to save variations, create a new yaml file and run this to register it

@orca.step()
def rh_cl_estimate_NEW(craigslist, aggregations):
    return utils.hedonic_estimate("rrh_NEW.yaml", craigslist, aggregations)

orca.run(["rrh_estimate_NEW"])

### 4. Compare to sales hedonic

In [7]:
orca.run(["rsh_estimate"])

Running step 'rsh_estimate'
                              OLS Regression Results                              
Dep. Variable:     np.log(price_per_sqft)   R-squared:                       0.376
Model:                                OLS   Adj. R-squared:                  0.376
Method:                     Least Squares   F-statistic:                     7097.
Date:                    Sun, 16 Aug 2015   Prob (F-statistic):               0.00
Time:                            17:15:11   Log-Likelihood:                -34553.
No. Observations:                  105997   AIC:                         6.913e+04
Df Residuals:                      105987   BIC:                         6.922e+04
Df Model:                               9                                         
Covariance Type:                nonrobust                                         
                                   coef    std err          t      P>|t|      [95.0% Conf. Int.]
---------------------------------------------